In [18]:
import pandas as pd
import numpy as np

In [19]:
df = pd.read_csv('data/AirlinesTweets.csv')
# df = pd.read_csv('data/sentiment140.csv', encoding='ISO-8859-1', names=names) ##use it for other datasets

In [20]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [21]:
df['sentiment'] = df.airline_sentiment.factorize()[0]
df = df.drop('airline_sentiment', axis=1)
df.head()

,tweet_id,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,sentiment
0,570306133677760513,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),0
1,570301130888122368,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),1
2,570301083672813571,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),0
3,570301031407624196,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),2
4,570300817074462722,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),2


In [22]:
df_copy = df.copy()
df = df[['text', 'sentiment']]
df.head()

,text,sentiment
0,@VirginAmerica What @dhepburn said.,0
1,@VirginAmerica plus you've added commercials t...,1
2,@VirginAmerica I didn't today... Must mean I n...,0
3,@VirginAmerica it's really aggressive to blast...,2
4,@VirginAmerica and it's a really big bad thing...,2


In [23]:
## looking at the lenth of each tweets
df['pre_clean_len'] = [len(t) for t in df.text]
df.head()

,text,sentiment,pre_clean_len
0,@VirginAmerica What @dhepburn said.,0,35
1,@VirginAmerica plus you've added commercials t...,1,72
2,@VirginAmerica I didn't today... Must mean I n...,0,71
3,@VirginAmerica it's really aggressive to blast...,2,126
4,@VirginAmerica and it's a really big bad thing...,2,55


In [29]:
## cleaning unnecessary values from the tweets

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer

tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not", "you've":"you have"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def tweet_cleaner_updated(text):
    soup = BeautifulSoup(text, 'html.parser')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    lower_case = stripped.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()

In [30]:
# testing = df.text[:100]
# testing.shape

# test_result = []
# for t in testing:
#     test_result.append(tweet_cleaner_updated(t))
# test_result[:10]

In [31]:
print("Cleaning and parsing the tweets...\n")
clean_tweet_texts = []
for i in range(len(df)):
    if( (i+1)%10000 == 0 ):
        print("Tweets %d of %d has been processed" % ( i+1, len(df) ))                                                                    
    clean_tweet_texts.append(tweet_cleaner_updated(df['text'][i]))

Cleaning and parsing the tweets...

Tweets 10000 of 14640 has been processed


In [32]:
clean_df = pd.DataFrame(clean_tweet_texts,columns=['text'])
clean_df['target'] = df.sentiment
clean_df.head()

,text,target
0,what said,0
1,plus you have added commercials to the experie...,1
2,did not today must mean need to take another trip,0
3,it really aggressive to blast obnoxious entert...,2
4,and it really big bad thing about it,2


In [33]:
new_df = df_copy.drop(['sentiment', 'text'], axis=1)
new_df['text'] = clean_df.text
new_df['target'] = clean_df.target
new_df.head()

,tweet_id,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,tweet_coord,tweet_created,tweet_location,user_timezone,text,target
0,570306133677760513,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),what said,0
1,570301130888122368,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),plus you have added commercials to the experie...,1
2,570301083672813571,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),did not today must mean need to take another trip,0
3,570301031407624196,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),it really aggressive to blast obnoxious entert...,2
4,570300817074462722,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),and it really big bad thing about it,2


In [17]:
new_df.to_csv('data/airlines_tweets_cleaned.csv', encoding='utf-8', index=None)